# Model New Data

In [1]:
import numpy as np
import pandas as pd
import acquire
import prepare
import model
from env import api_key
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import auc
from sklearn.metrics import plot_roc_curve
from sklearn.metrics import roc_curve
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.svm import SVC
from catboost import CatBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from mlxtend.classifier import EnsembleVoteClassifier
#validation & tuning
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import StandardScaler
cv=KFold(n_splits = 5, random_state=123,shuffle=True)

# Using Data At 10 Minute Timeframe

In [2]:
#Load final_10 csv file
data = pd.read_csv('../master10_csv/final_10.csv', index_col = [0])
data

,airdragon_team100,assistsplayer_1,assistsplayer_10,assistsplayer_2,assistsplayer_3,assistsplayer_4,assistsplayer_5,assistsplayer_6,assistsplayer_7,assistsplayer_8,...,xp_4,xp_5,xp_6,xp_7,xp_8,xp_9,chemtechdragon_team200,riftherald_team200,airdragon_team200,waterdragon_team200
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,3033.0,2297.0,3978.0,3193.0,4131.0,2971.0,NaN,NaN,NaN,NaN
1,0.0,1.0,1.0,2.0,2.0,2.0,2.0,0.0,1.0,2.0,...,2907.0,2719.0,2691.0,3758.0,4331.0,3662.0,NaN,NaN,NaN,NaN
2,0.0,1.0,2.0,1.0,4.0,5.0,7.0,1.0,3.0,2.0,...,3192.0,3234.0,4260.0,3189.0,4858.0,2538.0,NaN,NaN,NaN,NaN
3,0.0,0.0,2.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,...,2921.0,2417.0,4801.0,4457.0,3837.0,2916.0,1.0,NaN,NaN,NaN
4,1.0,0.0,2.0,0.0,1.0,0.0,1.0,0.0,3.0,0.0,...,3005.0,2671.0,4865.0,3579.0,4496.0,2644.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4858,0.0,1.0,1.0,2.0,2.0,1.0,4.0,0.0,1.0,2.0,...,3365.0,2670.0,4031.0,3851.0,2589.0,3621.0,NaN,NaN,NaN,NaN
4859,0.0,3.0,1.0,1.0,1.0,2.0,0.0,2.0,2.0,1.0,...,3119.0,4443.0,3667.0,4158.0,2940.0,3527.0,NaN,NaN,NaN,NaN
4860,0.0,2.0,1.0,0.0,1.0,0.0,2.0,0.0,1.0,2.0,...,4471.0,2958.0,4988.0,4474.0,2615.0,4185.0,NaN,NaN,NaN,NaN
4861,0.0,0.0,4.0,2.0,1.0,0.0,2.0,0.0,5.0,1.0,...,3347.0,1972.0,5326.0,2685.0,3750.0,3402.0,NaN,NaN,NaN,NaN


In [3]:
data.info(verbose = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4863 entries, 0 to 4862
Data columns (total 196 columns):
 #    Column                            Dtype  
---   ------                            -----  
 0    airdragon_team100                 float64
 1    assistsplayer_1                   float64
 2    assistsplayer_10                  float64
 3    assistsplayer_2                   float64
 4    assistsplayer_3                   float64
 5    assistsplayer_4                   float64
 6    assistsplayer_5                   float64
 7    assistsplayer_6                   float64
 8    assistsplayer_7                   float64
 9    assistsplayer_8                   float64
 10   assistsplayer_9                   float64
 11   baron_team100                     float64
 12   chemtechdragon_team100            float64
 13   currentGold_1                     float64
 14   currentGold_10                    float64
 15   currentGold_2                     float64
 16   currentGold_3         

In [4]:
#Now prepare it
train, test = prepare.prepare(data)
train.shape, test.shape

((3890, 229), (973, 229))

In [5]:
#Drop all columns that are categorical. They are not useful
cols_to_drop = train.select_dtypes('object').columns
cols_to_drop

Index(['gameMode', 'gameName', 'gameType', 'gameVersion', 'matchId'], dtype='object')

In [6]:
train = train.drop(columns = cols_to_drop)
test = test.drop(columns = cols_to_drop)

In [7]:
train.shape, test.shape

((3890, 224), (973, 224))

In [8]:
#Now separate it into X and y groups
X_train, y_train = train.drop(columns = ['winningTeam']), train.winningTeam
X_test, y_test = test.drop(columns = ['winningTeam']), train.winningTeam

In [9]:
#Create the dict of hyperparameters to optimize across
param_dict = {
    'max_depth': range(1, 16),
    'min_samples_leaf': range(1, 16)
}

In [10]:
#Now build the models
best_model = model.get_random_forest_models(X_train, y_train, param_dict, cv = 5)

KeyboardInterrupt: 

In [ ]:
#What were the most important features?
best_features = pd.DataFrame(best_model.feature_importances_, X_train.columns)
best_features.sort_values(by = 0, ascending = False).head(10)

## Catboost

In [10]:
cat_params = {"learning_rate":[0.005],
              "depth":[6,8]}

In [11]:
cat = CatBoostClassifier()

In [12]:
cat_grid = GridSearchCV(cat,cat_params,cv=cv)

In [ ]:
cat_grid.fit(X_train,y_train)

0:	learn: 0.6925882	total: 86.2ms	remaining: 1m 26s
1:	learn: 0.6921861	total: 95.3ms	remaining: 47.6s
2:	learn: 0.6916339	total: 103ms	remaining: 34.2s
3:	learn: 0.6912012	total: 110ms	remaining: 27.3s
4:	learn: 0.6906872	total: 117ms	remaining: 23.2s
5:	learn: 0.6902784	total: 123ms	remaining: 20.4s
6:	learn: 0.6898164	total: 130ms	remaining: 18.4s
7:	learn: 0.6893816	total: 136ms	remaining: 16.9s
8:	learn: 0.6890676	total: 142ms	remaining: 15.7s
9:	learn: 0.6885986	total: 149ms	remaining: 14.8s
10:	learn: 0.6881123	total: 156ms	remaining: 14s
11:	learn: 0.6877553	total: 163ms	remaining: 13.4s
12:	learn: 0.6874135	total: 169ms	remaining: 12.8s
13:	learn: 0.6870975	total: 175ms	remaining: 12.4s
14:	learn: 0.6865889	total: 182ms	remaining: 11.9s
15:	learn: 0.6860727	total: 188ms	remaining: 11.6s
16:	learn: 0.6854984	total: 194ms	remaining: 11.2s
17:	learn: 0.6851147	total: 201ms	remaining: 11s
18:	learn: 0.6846870	total: 208ms	remaining: 10.7s
19:	learn: 0.6842327	total: 214ms	remainin

167:	learn: 0.6376248	total: 1.27s	remaining: 6.31s
168:	learn: 0.6373545	total: 1.28s	remaining: 6.31s
169:	learn: 0.6371954	total: 1.29s	remaining: 6.3s
170:	learn: 0.6369357	total: 1.3s	remaining: 6.29s
171:	learn: 0.6366958	total: 1.31s	remaining: 6.29s
172:	learn: 0.6365007	total: 1.31s	remaining: 6.29s
173:	learn: 0.6363425	total: 1.32s	remaining: 6.28s
174:	learn: 0.6361714	total: 1.33s	remaining: 6.27s
175:	learn: 0.6359604	total: 1.34s	remaining: 6.26s
176:	learn: 0.6356847	total: 1.34s	remaining: 6.26s
177:	learn: 0.6355415	total: 1.35s	remaining: 6.25s
178:	learn: 0.6352901	total: 1.36s	remaining: 6.24s
179:	learn: 0.6351014	total: 1.37s	remaining: 6.23s
180:	learn: 0.6347808	total: 1.38s	remaining: 6.22s
181:	learn: 0.6345200	total: 1.38s	remaining: 6.21s
182:	learn: 0.6342366	total: 1.39s	remaining: 6.2s
183:	learn: 0.6339934	total: 1.4s	remaining: 6.19s
184:	learn: 0.6338221	total: 1.4s	remaining: 6.18s
185:	learn: 0.6335747	total: 1.41s	remaining: 6.17s
186:	learn: 0.633

330:	learn: 0.6071248	total: 2.46s	remaining: 4.97s
331:	learn: 0.6069643	total: 2.46s	remaining: 4.96s
332:	learn: 0.6068746	total: 2.47s	remaining: 4.95s
333:	learn: 0.6067534	total: 2.48s	remaining: 4.95s
334:	learn: 0.6066537	total: 2.49s	remaining: 4.94s
335:	learn: 0.6065236	total: 2.5s	remaining: 4.93s
336:	learn: 0.6063777	total: 2.5s	remaining: 4.93s
337:	learn: 0.6062585	total: 2.51s	remaining: 4.92s
338:	learn: 0.6060947	total: 2.52s	remaining: 4.91s
339:	learn: 0.6059948	total: 2.52s	remaining: 4.9s
340:	learn: 0.6057888	total: 2.53s	remaining: 4.89s
341:	learn: 0.6056049	total: 2.54s	remaining: 4.89s
342:	learn: 0.6054592	total: 2.55s	remaining: 4.88s
343:	learn: 0.6053335	total: 2.55s	remaining: 4.87s
344:	learn: 0.6051873	total: 2.56s	remaining: 4.86s
345:	learn: 0.6050285	total: 2.57s	remaining: 4.85s
346:	learn: 0.6048647	total: 2.58s	remaining: 4.85s
347:	learn: 0.6047393	total: 2.58s	remaining: 4.84s
348:	learn: 0.6046503	total: 2.59s	remaining: 4.83s
349:	learn: 0.6

507:	learn: 0.5834106	total: 3.65s	remaining: 3.53s
508:	learn: 0.5833124	total: 3.66s	remaining: 3.53s
509:	learn: 0.5831488	total: 3.66s	remaining: 3.52s
510:	learn: 0.5830425	total: 3.67s	remaining: 3.51s
511:	learn: 0.5829589	total: 3.68s	remaining: 3.51s
512:	learn: 0.5828334	total: 3.69s	remaining: 3.5s
513:	learn: 0.5826425	total: 3.69s	remaining: 3.49s
514:	learn: 0.5825370	total: 3.7s	remaining: 3.48s
515:	learn: 0.5824245	total: 3.71s	remaining: 3.48s
516:	learn: 0.5823183	total: 3.71s	remaining: 3.47s
517:	learn: 0.5821984	total: 3.72s	remaining: 3.46s
518:	learn: 0.5820258	total: 3.73s	remaining: 3.46s
519:	learn: 0.5819667	total: 3.73s	remaining: 3.45s
520:	learn: 0.5818184	total: 3.74s	remaining: 3.44s
521:	learn: 0.5816319	total: 3.75s	remaining: 3.43s
522:	learn: 0.5815191	total: 3.75s	remaining: 3.42s
523:	learn: 0.5813776	total: 3.76s	remaining: 3.42s
524:	learn: 0.5813109	total: 3.77s	remaining: 3.41s
525:	learn: 0.5811871	total: 3.78s	remaining: 3.4s
526:	learn: 0.5

684:	learn: 0.5630481	total: 4.83s	remaining: 2.22s
685:	learn: 0.5629448	total: 4.84s	remaining: 2.21s
686:	learn: 0.5628376	total: 4.85s	remaining: 2.21s
687:	learn: 0.5627179	total: 4.86s	remaining: 2.2s
688:	learn: 0.5626081	total: 4.86s	remaining: 2.19s
689:	learn: 0.5625169	total: 4.87s	remaining: 2.19s
690:	learn: 0.5623773	total: 4.88s	remaining: 2.18s
691:	learn: 0.5622767	total: 4.88s	remaining: 2.17s
692:	learn: 0.5621746	total: 4.89s	remaining: 2.17s
693:	learn: 0.5620644	total: 4.9s	remaining: 2.16s
694:	learn: 0.5618947	total: 4.9s	remaining: 2.15s
695:	learn: 0.5617691	total: 4.91s	remaining: 2.14s
696:	learn: 0.5616360	total: 4.92s	remaining: 2.14s
697:	learn: 0.5614958	total: 4.92s	remaining: 2.13s
698:	learn: 0.5613750	total: 4.93s	remaining: 2.12s
699:	learn: 0.5612701	total: 4.94s	remaining: 2.12s
700:	learn: 0.5612214	total: 4.94s	remaining: 2.11s
701:	learn: 0.5611186	total: 4.95s	remaining: 2.1s
702:	learn: 0.5609853	total: 4.96s	remaining: 2.09s
703:	learn: 0.56

...

66:	learn: 0.6512792	total: 1.4s	remaining: 19.6s
67:	learn: 0.6508159	total: 1.43s	remaining: 19.6s
68:	learn: 0.6501542	total: 1.45s	remaining: 19.6s
69:	learn: 0.6496239	total: 1.47s	remaining: 19.5s
70:	learn: 0.6490639	total: 1.49s	remaining: 19.5s
71:	learn: 0.6486292	total: 1.51s	remaining: 19.5s
72:	learn: 0.6480579	total: 1.53s	remaining: 19.5s
73:	learn: 0.6474535	total: 1.55s	remaining: 19.4s
74:	learn: 0.6470519	total: 1.57s	remaining: 19.4s
75:	learn: 0.6465450	total: 1.59s	remaining: 19.4s
76:	learn: 0.6460636	total: 1.61s	remaining: 19.3s
77:	learn: 0.6454330	total: 1.64s	remaining: 19.3s
78:	learn: 0.6449368	total: 1.66s	remaining: 19.3s
79:	learn: 0.6444495	total: 1.68s	remaining: 19.3s
80:	learn: 0.6438719	total: 1.7s	remaining: 19.3s
81:	learn: 0.6433641	total: 1.72s	remaining: 19.2s
82:	learn: 0.6428174	total: 1.74s	remaining: 19.2s
83:	learn: 0.6422753	total: 1.76s	remaining: 19.2s
84:	learn: 0.6417451	total: 1.78s	remaining: 19.2s
85:	learn: 0.6412088	total: 1.8s	

226:	learn: 0.5831007	total: 4.75s	remaining: 16.2s
227:	learn: 0.5827816	total: 4.78s	remaining: 16.2s
228:	learn: 0.5824567	total: 4.8s	remaining: 16.2s
229:	learn: 0.5822201	total: 4.82s	remaining: 16.1s
230:	learn: 0.5819741	total: 4.84s	remaining: 16.1s
231:	learn: 0.5815936	total: 4.86s	remaining: 16.1s
232:	learn: 0.5812572	total: 4.88s	remaining: 16.1s
233:	learn: 0.5810076	total: 4.9s	remaining: 16s
234:	learn: 0.5805836	total: 4.92s	remaining: 16s
235:	learn: 0.5803112	total: 4.94s	remaining: 16s
236:	learn: 0.5800258	total: 4.96s	remaining: 16s
237:	learn: 0.5797451	total: 4.98s	remaining: 16s
238:	learn: 0.5794437	total: 5s	remaining: 15.9s
239:	learn: 0.5790341	total: 5.03s	remaining: 15.9s
240:	learn: 0.5788337	total: 5.05s	remaining: 15.9s
241:	learn: 0.5784998	total: 5.07s	remaining: 15.9s
242:	learn: 0.5781430	total: 5.09s	remaining: 15.9s
243:	learn: 0.5778096	total: 5.11s	remaining: 15.8s
244:	learn: 0.5774455	total: 5.13s	remaining: 15.8s
245:	learn: 0.5770796	total

387:	learn: 0.5334657	total: 8.08s	remaining: 12.7s
388:	learn: 0.5332524	total: 8.1s	remaining: 12.7s
389:	learn: 0.5328803	total: 8.12s	remaining: 12.7s
390:	learn: 0.5326213	total: 8.14s	remaining: 12.7s
391:	learn: 0.5323911	total: 8.16s	remaining: 12.7s
392:	learn: 0.5320653	total: 8.18s	remaining: 12.6s
393:	learn: 0.5317049	total: 8.2s	remaining: 12.6s
394:	learn: 0.5312760	total: 8.22s	remaining: 12.6s
395:	learn: 0.5311203	total: 8.24s	remaining: 12.6s
396:	learn: 0.5307396	total: 8.27s	remaining: 12.6s
397:	learn: 0.5305216	total: 8.29s	remaining: 12.5s
398:	learn: 0.5303690	total: 8.31s	remaining: 12.5s
399:	learn: 0.5301626	total: 8.33s	remaining: 12.5s
400:	learn: 0.5298089	total: 8.35s	remaining: 12.5s
401:	learn: 0.5295287	total: 8.37s	remaining: 12.5s
402:	learn: 0.5292063	total: 8.39s	remaining: 12.4s
403:	learn: 0.5289991	total: 8.41s	remaining: 12.4s
404:	learn: 0.5287497	total: 8.43s	remaining: 12.4s
405:	learn: 0.5284847	total: 8.45s	remaining: 12.4s
406:	learn: 0.

547:	learn: 0.4936903	total: 11.5s	remaining: 9.45s
548:	learn: 0.4934164	total: 11.5s	remaining: 9.43s
549:	learn: 0.4932277	total: 11.5s	remaining: 9.41s
550:	learn: 0.4929373	total: 11.5s	remaining: 9.39s
551:	learn: 0.4927050	total: 11.5s	remaining: 9.37s
552:	learn: 0.4923818	total: 11.6s	remaining: 9.35s
553:	learn: 0.4921530	total: 11.6s	remaining: 9.33s
554:	learn: 0.4919456	total: 11.6s	remaining: 9.31s
555:	learn: 0.4917079	total: 11.6s	remaining: 9.29s
556:	learn: 0.4915309	total: 11.7s	remaining: 9.27s
557:	learn: 0.4912227	total: 11.7s	remaining: 9.24s
558:	learn: 0.4910985	total: 11.7s	remaining: 9.23s
559:	learn: 0.4908636	total: 11.7s	remaining: 9.21s
560:	learn: 0.4905911	total: 11.7s	remaining: 9.19s
561:	learn: 0.4903521	total: 11.8s	remaining: 9.16s
562:	learn: 0.4901092	total: 11.8s	remaining: 9.14s
563:	learn: 0.4899051	total: 11.8s	remaining: 9.12s
564:	learn: 0.4896749	total: 11.8s	remaining: 9.1s
565:	learn: 0.4894366	total: 11.8s	remaining: 9.08s
566:	learn: 0

706:	learn: 0.4598986	total: 14.8s	remaining: 6.14s
707:	learn: 0.4598085	total: 14.8s	remaining: 6.12s
708:	learn: 0.4595393	total: 14.8s	remaining: 6.09s
709:	learn: 0.4594027	total: 14.9s	remaining: 6.07s
710:	learn: 0.4590258	total: 14.9s	remaining: 6.05s
711:	learn: 0.4588350	total: 14.9s	remaining: 6.03s
712:	learn: 0.4586884	total: 14.9s	remaining: 6.01s
713:	learn: 0.4585232	total: 14.9s	remaining: 5.99s
714:	learn: 0.4583859	total: 15s	remaining: 5.97s
715:	learn: 0.4582323	total: 15s	remaining: 5.95s
716:	learn: 0.4581642	total: 15s	remaining: 5.92s
717:	learn: 0.4579725	total: 15s	remaining: 5.9s
718:	learn: 0.4577190	total: 15.1s	remaining: 5.88s
719:	learn: 0.4575506	total: 15.1s	remaining: 5.86s
720:	learn: 0.4572879	total: 15.1s	remaining: 5.84s
721:	learn: 0.4571620	total: 15.1s	remaining: 5.82s
722:	learn: 0.4569838	total: 15.1s	remaining: 5.8s
723:	learn: 0.4567808	total: 15.2s	remaining: 5.78s
724:	learn: 0.4566158	total: 15.2s	remaining: 5.75s
725:	learn: 0.4564564	

In [ ]:
cat_grid.best_params_

In [ ]:
cat = CatBoostClassifier(learning_rate=0.005,depth=16,verbose=False)

## Logistic Regression

In [ ]:
log_params = {"penalty":["l1","l2","elasticnet"],
              "solver":['newton-cg', 'lbfgs', 'sag', 'saga'],
             "C":[1,5,10,20,30,50,100],
             "warm_start":[True,False]}

In [ ]:
logreg=LogisticRegression()

In [ ]:
log_grid = GridSearchCV(logreg,log_params,cv=cv,verbose=2,n_jobs=-1)

In [ ]:
log_grid.fit(X_train,y_train)

## SVC

In [ ]:
svc_params={'kernel':['poly','sigmoid'],
           'C':[1,10,20,50,100]}

In [ ]:
svc=SVC()

In [ ]:
svc_grid=GridSearchCV(svc,svc_params,cv=cv,verbose=2,n_jobs=-1)

In [ ]:
svc_grid.fit(X_train,y_train)

In [ ]:
svc_grid.best_params_

In [ ]:
svc=SVC(kernel="poly",probability=True,C=20)

In [ ]:
np.mean(cross_val_score(svc,X_train,y_train,cv=cv,scoring="accuracy",n_jobs=-1))

## ADA Boost

In [ ]:
ada_params={"learning_rate":[0.05,0.1,0.2,0.5,1],
           "algorithm":['SAMME', 'SAMME.R'],
           "n_estimators":[50,100,150,500]}

In [ ]:
ada = AdaBoostClassifier()

In [ ]:
ada_grid = GridSearchCV(ada,ada_params,cv=cv,verbose=2)

In [ ]:
ada_grid.fit(X_train,y_train)

In [ ]:
ada_grid.best_params_

In [ ]:
ada_best_params = {"learning_rate":0.05,
                  "algorithm":"SAMME.R",
                  "n_estimators":150}


In [ ]:
ada=AdaBoostClassifier(**ada_best_params)

In [ ]:
np.mean(cross_val_score(ada,X_train,y_train,cv=cv,scoring="accuracy",n_jobs=-1))

## RFE

In [ ]:
rdf_params={"criterion":["entropy"],#gini
           "max_depth":[8], #2,4,6
           "min_samples_split":[4],#2,6
           "max_features":["sqrt"]} #log2

In [ ]:
rdf=RandomForestClassifier()

In [ ]:
rdf_grid=GridSearchCV(rdf,rdf_params,cv=cv,verbose=2,n_jobs=-1)

In [ ]:
rdf_grid.fit(X_train,y_train)

In [ ]:
rdf_grid.best_params_

In [ ]:
rdf=RandomForestClassifier(criterion="entropy",max_depth=8,max_features="sqrt",min_samples_split=4)

In [ ]:
np.mean(cross_val_score(rdf,X_train,y_train,cv=cv,scoring="accuracy",n_jobs=-1))

In [ ]:
eclf = EnsembleVoteClassifier(clfs=[cat,logreg, svc,ada,rdf], weights=[1,1,1,1,1,1,1])
labels = ['CatBoost','Logistic Regression', 'SVC','AdaBoost',"Random Forest",'Ensemble']
for clf, label in zip([cat,logreg, svc, ada, rdf,eclf], labels):

    scores = cross_val_score(clf, X_train,y_train, 
                             cv=cv, 
                             scoring='accuracy',
                             n_jobs=-1)

    print("[%s] Accuracy: %0.6f (+/- %0.6f) Best: %0.6f " 
          % (label,scores.mean(), scores.std(), scores.max()))

# Can We Remove Individual Stats and Maintain or Improve Model Accuracy?

In [ ]:
#Only keep columns that have the word 'team' in their name
cols_to_keep = X_train.columns[X_train.columns.str.contains('Team')]
cols_to_keep

In [ ]:
X_train_team_data = X_train.loc[:, cols_to_keep]
X_train_team_data

In [ ]:
#Build the models
best_model = model.get_random_forest_models(X_train_team_data, y_train, param_dict, cv = 5)

In [ ]:
#What were the most important features?
best_features = pd.DataFrame(best_model.feature_importances_, X_train_team_data.columns)
best_features.sort_values(by = 0, ascending = False).head(10)

# Can We Keep Only One Team's Data and Maintain or Improve Model Accuracy?

In [ ]:
cols_to_keep = X_train_team_data.columns[X_train_team_data.columns.str.contains('Blue')]
cols_to_keep

In [ ]:
X_train_blue_team = X_train_team_data.loc[:, cols_to_keep]
X_train_blue_team

In [ ]:
#Build the models
best_model = model.get_random_forest_models(X_train_blue_team, y_train, param_dict, cv = 5)

In [ ]:
y_train.value_counts(normalize = True)

In [ ]:
#What were the most important features?
best_features = pd.DataFrame(best_model.feature_importances_, X_train_blue_team.columns)
best_features.sort_values(by = 0, ascending = False).head(10)

# Can We Only Use Individual Stats and Maintain or Improve Model Accuracy?

In [ ]:
cols_to_keep = X_train.columns[(X_train.columns.str.contains('Team') == False)& (X_train.columns.str.contains('dragon') == False)]
cols_to_keep

In [ ]:
X_train_players = X_train.loc[:, cols_to_keep]
X_train_players

In [ ]:
#Build the models
best_model = model.get_random_forest_models(X_train_players, y_train, param_dict, cv = 5)

In [ ]:
#What were the most important features?
best_features = pd.DataFrame(best_model.feature_importances_, X_train_players.columns)
best_features.sort_values(by = 0, ascending = False).head(10)

# Will a KNN Model Improve Accuracy?

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV

In [ ]:
#First need to scale the data
#Create the scaler
scaler = MinMaxScaler()

#Fit and transform on train data
X_train_scaled = scaler.fit_transform(X_train)

In [ ]:
#Build a function for KNN
def get_KNN_models(X_train_scaled, y_train, param_dict, cv = 5):
    """
    This function takes in scaled data and builds an optimized KNN classification model. 
    It will use the parameters specified in the param_dict to optimizie across.
    
    This function utilizes GridSearchCV.
    
    This function returns the best model and prints out its parameters and mean
    cross-validated accuracy.
    """
    #Create the KNN model
    clf = KNeighborsClassifier()
    
    #Create the GridSearchCV object
    grid = GridSearchCV(clf, param_dict, cv)
    
    #Fit the GridSearchCV object
    grid.fit(X_train_scaled, y_train)
    
    #Print the best model's score and parameters
    print('Mean Cross-Validated Accuracy: ', round(grid.best_score_, 4))
    print('Num Neighbors: ', grid.best_params_['n_neighbors'])
    print('Weights: ', grid.best_params_['weights'])
    
    #Return the best model
    return grid.best_estimator_

In [ ]:
#Create a param_dict for KNN model
param_dict = {
    'n_neighbors': range(200, 500),
    'weights': ['uniform', 'distance']
}

In [ ]:
#Test the function. Run it on the full X_train_scaled data set (Team and Player data)
best_model = get_KNN_models(X_train_scaled, y_train, param_dict, cv = 5)

# Can We Use Only Team Data and Maintain or Improve KNN Model Accuracy?

In [ ]:
#Scale the data
X_train_team_data_scaled = scaler.fit_transform(X_train_team_data)

In [ ]:
#build the models
best_model = get_KNN_models(X_train_team_data_scaled, y_train, param_dict, cv = 5)

# Can We Use Only A Single Team's Data and Maintain or Improve KNN Model Accuracy?

In [ ]:
#Scale the data
X_train_blue_team_scaled = scaler.fit_transform(X_train_blue_team)

In [ ]:
#Build the models
best_model = get_KNN_models(X_train_blue_team_scaled, y_train, param_dict, cv = 5)

# Can We Only Use Individual Stats and Maintain or Improve KNN Model Accuracy?

In [ ]:
#Scale the data
X_train_players_scaled = scaler.fit_transform(X_train_players)

In [ ]:
#Build the models
best_model = get_KNN_models(X_train_players_scaled, y_train, param_dict, cv = 5)

# Try Using Josh C's Suggested Features for Both Random Forest and KNN Models

In [ ]:
suggested_features = [
    'xp_1',
    'killsplayer_2',
    'killsplayer_4',
    'assistsplayer_5'
]

In [ ]:
X_train_suggested = X_train.loc[:, suggested_features]
X_train_suggested

__Random Forest Classifier__

In [ ]:
rf_dict = {
    'max_depth': range(1, 16),
    'min_samples_leaf': range(1, 16)
}

In [ ]:
#Build the random forest models
best_model = model.get_random_forest_models(X_train_suggested, y_train, rf_dict, cv = 5)

__KNN Classifier__

In [ ]:
#Scale the data
X_train_suggested_scaled = scaler.fit_transform(X_train_suggested)

In [ ]:
knn_dict = {
    'n_neighbors': range(200, 500),
    'weights': ['uniform', 'distance']
}

In [ ]:
#Build the KNN models
best_model = get_KNN_models(X_train_suggested_scaled, y_train, knn_dict, cv = 5)

# Can We Add Blue Team Stats and Improve Model Accuracy?

In [ ]:
#Update cols_to_keep
cols_to_keep = list(X_train_blue_team.columns)
cols_to_keep

In [ ]:
cols_to_keep.extend(suggested_features)

In [ ]:
cols_to_keep

In [ ]:
X_train_blue_team_suggested = X_train.loc[:, cols_to_keep]
X_train_blue_team_suggested

__Random Forest Classifier__

In [ ]:
#Build random forest models
best_model = model.get_random_forest_models(X_train_blue_team_suggested, y_train, rf_dict, cv = 5)

In [ ]:
#What were the most important features?
best_features = pd.DataFrame(best_model.feature_importances_, X_train_blue_team_suggested.columns)
best_features.sort_values(by = 0, ascending = False).head(10)

__KNN Classifier__

In [ ]:
#Scale the data
X_train_blue_team_suggested_scaled = scaler.fit_transform(X_train_blue_team_suggested)

In [ ]:
#Build the KNN models
best_model = get_KNN_models(X_train_blue_team_suggested_scaled, y_train, knn_dict, cv = 5)

# Can an AdaBoost Classifier Improve Model Accuracy?

In [ ]:
#Set the params to optimize across
adaBoost_params = {
    'n_estimators': range(25, 101),
    'learning_rate': range(1, 11)
}

In [ ]:
#Construct the model to use with adaBoost. Try Single Decision Tree
from sklearn.tree import DecisionTreeClassifier

model_to_boost = DecisionTreeClassifier(random_state = 123)

In [ ]:
best_model = model.get_adaBoosted_model(X_train_blue_team, y_train, model_to_boost, adaBoost_params, cv = 5)